In [11]:
import sys
sys.path.append("../")
from src.utils.logger import Logger
logger = Logger(__name__)
from src.RAG.embedded_data import emmbeded_data_folder, EmmbededData
from src.RAG.qdrant_vectordb import QdrantVectorDB
import os
os.environ["QDRANT_URL"] = "localhost:6333"

In [12]:
emmbeded_data_folder("../data/RAG", "document")

2025-06-21 17:19:55,217 - src.RAG.embedded_data - INFO - ✅ Emmbeding data from ../data/RAG/Giao an theo Tuan Lop 4 Giao an Tuan 28  Lop 4.docx to collection document
INFO:src.RAG.embedded_data:✅ Emmbeding data from ../data/RAG/Giao an theo Tuan Lop 4 Giao an Tuan 28  Lop 4.docx to collection document
2025-06-21 17:19:55,534 - src.RAG.embedded_data - INFO - ✅ Loaded 121 documents from ../data/RAG/Giao an theo Tuan Lop 4 Giao an Tuan 28  Lop 4.docx
INFO:src.RAG.embedded_data:✅ Loaded 121 documents from ../data/RAG/Giao an theo Tuan Lop 4 Giao an Tuan 28  Lop 4.docx
2025-06-21 17:19:55,652 - src.RAG.qdrant_vectordb - INFO - ✅ 
            QdrantVectorDB initialized with:
            - URL: localhost:6333
            - Vector name: dense
            - Model embedding: BAAI/bge-small-en
        
INFO:src.RAG.qdrant_vectordb:✅ 
            QdrantVectorDB initialized with:
            - URL: localhost:6333
            - Vector name: dense
            - Model embedding: BAAI/bge-small-en
     

In [3]:
doc = EmmbededData().read_docx("../data/RAG/Giao an theo Tuan Lop 4 Giao an Tuan 28  Lop 4.docx")

In [13]:
splitt = EmmbededData().split_text(doc)

In [5]:
from qdrant_client import QdrantClient, models

# 1. Connect to Qdrant server
client = QdrantClient("http://localhost:6333")

In [ ]:
client.get_collections().model_dump().get("collections")

[{'name': 'dense_multivector_demo'},
 {'name': 'document'},
 {'name': 'test_collection'}]

In [55]:
def  check_collection(collection_name:str):
    try:
        collections = client.get_collections().model_dump().get("collections")
        if {'name':collection_name} in collections: #type:ignore
            logger.info(f"Collection `{collection_name}` already exists")
            return True
        logger.warning(f"Collection `{collection_name}` does not exist")
        return False
    except Exception as e:
        logger.error(f"❌ Failed to check collection: {e}")
        return False

check_collection("document")


2025-06-21 16:19:03,123 - __main__ - INFO - ✅ Collection `document` already exists
INFO:__main__:✅ Collection `document` already exists


True

In [38]:

query_text = "Tuần 28"
dense_documents = [
    models.Document(text=doc, model="BAAI/bge-small-en")
    for doc in splitt
]
dense_query = models.Document(text=query_text, model="BAAI/bge-small-en")
print(dense_query)

text='Tuần 28' model='BAAI/bge-small-en' options=None


In [18]:
collection_name = "dense_multivector_demo"
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": models.VectorParams(
            size=384,
            distance=models.Distance.COSINE
            # Leave HNSW indexing ON for dense
        ),
        "colbert": models.VectorParams(
            size=128,
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM
            ),
            hnsw_config=models.HnswConfigDiff(m=0)  # Disable HNSW for reranking
        )
    }
)

True

In [56]:
query_text = "Tuần 28"
dense_documents = [
    models.Document(text=doc, model="BAAI/bge-small-en")
    for doc in splitt
]
dense_query = models.Document(text=query_text, model="BAAI/bge-small-en")
print(dense_query)

points = [
    models.PointStruct(
        id=i,
        vector={
            "dense": dense_documents[i],
        },
        payload={"text": splitt[i]}
    ) for i in range(len(splitt))
]
client.upsert(collection_name="dense_multivector_demo", points=points)

text='Tuần 28' model='BAAI/bge-small-en' options=None


UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [39]:
results = client.query_points(
    collection_name="dense_multivector_demo",
    prefetch=models.Prefetch(
        query=dense_query,
        using="dense",
    ),
    query=dense_query,
    using="dense",
    limit=3,
    with_payload=True
)

In [60]:
results.model_dump().get("points")


[{'id': 0,
  'version': 2,
  'score': 0.8650309,
  'payload': {'text': 'TUẦN 28\n(Từ ngày 31/03 đến ngày 04/04/2025)\nThứ hai ngày 31 tháng 03 năm 2025\nBUỔI SÁNG:\nTiết 1: \t\t\t\t          CHÀO CỜ\t\tPPCT: 28\n TUẦN 28\n                                             HOẠT ĐỘNG TRẢI NGHIỆM                                  PPCT: 82\nHƯỞNG ỨNG PHONG TRÀO “MÔI TRƯỜNG XANH – CUỘC SỐNG XANH”.\n*Hoạt động trải nghiệm: Hưởng ứng phong trào “Môi trường xanh – Cuộc sống xanh”.\nA. Yêu cầu cần đạt:\nGiới thiệu được với bạn bè, người thân về cảnh quan thiên nhiên ở địa phương.'},
  'vector': None,
  'shard_key': None,
  'order_value': None},
 {'id': 120,
  'version': 2,
  'score': 0.85663694,
  'payload': {'text': 'II. SHTT: Tuần 28 (20’)\nIII. Vui chơi, giải trí: ( 5’ )\nHát tập thể'},
  'vector': None,
  'shard_key': None,
  'order_value': None},
 {'id': 116,
  'version': 2,
  'score': 0.8556199,
  'payload': {'text': 'TUẦN 28\n\t\t\t\t\t\t\t\t\t\t\tTGDK: 35              \nI. Hoạt động trải nghiệ

In [15]:
QdrantVectorDB().query("dense_multivector_demo", "Tuần 28", 10)

2025-06-21 17:22:05,887 - src.RAG.qdrant_vectordb - INFO - ✅ 
            QdrantVectorDB initialized with:
            - URL: localhost:6333
            - Vector name: dense
            - Model embedding: BAAI/bge-small-en
        
INFO:src.RAG.qdrant_vectordb:✅ 
            QdrantVectorDB initialized with:
            - URL: localhost:6333
            - Vector name: dense
            - Model embedding: BAAI/bge-small-en
        
2025-06-21 17:22:05,892 - src.RAG.qdrant_vectordb - INFO - ✅ Collection `dense_multivector_demo` already exists
INFO:src.RAG.qdrant_vectordb:✅ Collection `dense_multivector_demo` already exists
2025-06-21 17:22:07,039 - src.RAG.qdrant_vectordb - INFO - ✅ ✅ Retrieved results from collection dense_multivector_demo
INFO:src.RAG.qdrant_vectordb:✅ ✅ Retrieved results from collection dense_multivector_demo


[{'id': 0,
  'version': 3,
  'score': 0.8650309,
  'payload': {'text': 'TUẦN 28\n(Từ ngày 31/03 đến ngày 04/04/2025)\nThứ hai ngày 31 tháng 03 năm 2025\nBUỔI SÁNG:\nTiết 1: \t\t\t\t          CHÀO CỜ\t\tPPCT: 28\n TUẦN 28\n                                             HOẠT ĐỘNG TRẢI NGHIỆM                                  PPCT: 82\nHƯỞNG ỨNG PHONG TRÀO “MÔI TRƯỜNG XANH – CUỘC SỐNG XANH”.\n*Hoạt động trải nghiệm: Hưởng ứng phong trào “Môi trường xanh – Cuộc sống xanh”.\nA. Yêu cầu cần đạt:\nGiới thiệu được với bạn bè, người thân về cảnh quan thiên nhiên ở địa phương.'},
  'vector': None,
  'shard_key': None,
  'order_value': None},
 {'id': 120,
  'version': 3,
  'score': 0.85663694,
  'payload': {'text': 'II. SHTT: Tuần 28 (20’)\nIII. Vui chơi, giải trí: ( 5’ )\nHát tập thể'},
  'vector': None,
  'shard_key': None,
  'order_value': None},
 {'id': 116,
  'version': 3,
  'score': 0.8556199,
  'payload': {'text': 'TUẦN 28\n\t\t\t\t\t\t\t\t\t\t\tTGDK: 35              \nI. Hoạt động trải nghiệ